In [ ]:
try:
    import evidently
except:
    !pip install git+https://github.com/evidentlyai/evidently.git

In [ ]:
import pandas as pd
import numpy as np

import io
import os
import zipfile

import requests

In [ ]:
!pip install implicit

# load data

In [ ]:
content = requests.get("http://files.grouplens.org/datasets/movielens/ml-100k.zip").content

with zipfile.ZipFile(io.BytesIO(content)) as arc:
  train = arc.read("ml-100k/ua.base").decode().split("\n")
  test = arc.read("ml-100k/ua.test").decode().split("\n")
  movies = arc.read("ml-100k/u.item").decode(encoding='latin-1').split("\n")
  users = arc.read("ml-100k/u.user").decode(encoding='latin-1').split("\n")

In [ ]:
columns = ['user_id', 'movie_id', 'rating', 'timestamp']

data = [[x for x in e.split('\t')] for e in train]
train = pd.DataFrame(data, columns=columns).dropna().astype(int)

data = [[x for x in e.split('\t')] for e in test]
test = pd.DataFrame(data, columns=columns).dropna().astype(int)

columns = ['user_id', 'age', 'gender', 'occupation', 'zip_code']

data = [[x for x in e.split('|')] for e in users]
users = pd.DataFrame(data, columns=columns).dropna().astype({'user_id': int, 'age': int})

genres = ['unknown', 'action', 'adventure', 'animation', 'children', 'comedy', 'crime', 'documentary', 'drama', 'fantasy', 'noir',
          'horror', 'musical', 'mystery', 'romance', 'sci-fi', 'thriller', 'war', 'western']
columns = ['movie_id', 'title', 'year', '-', 'url'] + genres
data = [[x for x in e.split('|')] for e in movies]
movies = pd.DataFrame(data, columns=columns).dropna().astype({'movie_id': int})
movies.drop(columns=['-', 'url'], inplace=True)
movies[genres] = movies[genres].astype(int)
movies['moive_age'] = (pd.to_datetime(movies.year).max() - pd.to_datetime(movies.year)).dt.days / 365

# Utils

In [ ]:
def transform_predictions(k, user_ids, item_ids):
  return pd.DataFrame(
      data=np.c_[np.repeat(user_ids, k), item_ids.flatten(), [i + 1 for i in range(k)] * len(user_ids)],
      columns=['user_id', 'movie_id', 'rank']
  )


def prepare_prediction_df(k, user_ids, item_ids, true):
  preds = transform_predictions(k, user_ids, item_ids)
  preds = preds.merge(true, on=['user_id', 'movie_id'], how='outer')
  preds['rank'] = preds.groupby('user_id')['rank'].transform(lambda x: x.fillna(x.max() + 1))
  return preds


def get_embeddings(model, movies_list, users_list, factors):
  item_factors = pd.DataFrame(
      data=np.column_stack((movies_list, model.item_factors)),
      columns=['movie_id'] + [f'item_factor_{i+1}' for i in range(factors)]
  )
  user_factors = pd.DataFrame(
      data=np.column_stack((users_list, model.user_factors)),
      columns=['user_id'] + [f'user_factor_{i+1}' for i in range(factors)]
  )
  return item_factors, user_factors


def get_full_df(df, item_factors, user_factors):
  df = df.merge(movies, on=['movie_id'], how='left')
  df = df.merge(users, on=['user_id'], how='left')
  df = df.merge(item_factors, on=['movie_id'], how='left')
  df = df.merge(user_factors, on=['user_id'], how='left')
  return df

# Train model

Let's get predictions from two models - ALS model and most common item recommender

In [ ]:
from implicit.cpu.als import AlternatingLeastSquares
from scipy.sparse import csr_matrix
pivot_table = train.pivot_table(index=['user_id'], columns=['movie_id'], values="rating").fillna(0)

als_model = AlternatingLeastSquares(factors=20, iterations=5, random_state=0)
als_model.fit(csr_matrix(pivot_table))

In [ ]:
ids, scores = als_model.recommend(test.user_id.unique() - 1, csr_matrix(pivot_table.loc[test.user_id.unique()]), N=30, filter_already_liked_items=True)
als_df = prepare_prediction_df(30, test.user_id.unique(), ids, test)

In [ ]:
most_popular_top = list(train.movie_id.value_counts()[:30].index)
rec_array = np.array([most_popular_top] * len(test.user_id.unique()))
most_popular_df = prepare_prediction_df(30, test.user_id.unique(), rec_array, test)

In [ ]:
item_factors, user_factors = get_embeddings(als_model, pivot_table.columns, pivot_table.index, 20)
als_df = get_full_df(als_df, item_factors, user_factors)
most_popular_df = get_full_df(most_popular_df, item_factors, user_factors)
train = get_full_df(train, item_factors, user_factors)

In [ ]:
item_features = [f'item_factor_{i+1}' for i in range(20)]

# Metrics

In [ ]:
from evidently.metrics import PrecisionTopKMetric
from evidently.metrics import RecallTopKMetric
from evidently.metrics import FBetaTopKMetric
from evidently.metrics import MAPKMetric
from evidently.metrics import NDCGKMetric
from evidently.metrics import DiversityMetric
from evidently.metrics import ItemBiasMetric
from evidently.metrics import NoveltyMetric
from evidently.metrics import PersonalizationMetric
from evidently.metrics import PopularityBias
from evidently.metrics import SerendipityMetric
from evidently.metrics import UserBiasMetric
from evidently.metrics import HitRateKMetric
from evidently.metrics import MRRKMetric
from evidently.metrics import RecCasesTable

from evidently.pipeline.column_mapping import ColumnMapping
from evidently.report import Report

In [ ]:
report = Report(metrics=[
    PrecisionTopKMetric(k=5),
    RecallTopKMetric(k=5),
    FBetaTopKMetric(k=5),
    MAPKMetric(k=5),
    NDCGKMetric(k=5),
    MRRKMetric(k=5),
    HitRateKMetric(k=5),
    DiversityMetric(k=5, item_features=item_features),
    NoveltyMetric(k=5),
    PersonalizationMetric(k=5),
    SerendipityMetric(k=5, item_features=item_features),
    PopularityBias(k=5),
    ItemBiasMetric(k=5, column_name='moive_age'),
    ItemBiasMetric(k=5, column_name='crime'),
    UserBiasMetric(column_name='age'),
    UserBiasMetric(column_name='gender'),
    RecCasesTable(display_features=['action', 'adventure', 'animation'], item_num=10)


])
column_mapping = ColumnMapping(recommendations_type='rank', target='rating', prediction='rank', item_id='title', user_id='user_id')
report.run(
    reference_data=most_popular_df.dropna(subset=['title', 'user_id']).fillna(0),
    current_data=als_df.dropna(subset=['title', 'user_id']).fillna(0),
    column_mapping=column_mapping,
    additional_data={'current_train_data': train.dropna(subset=['title', 'user_id'])}
  )
report

# Metric Preset

In [ ]:
from evidently.metric_preset import RecsysPreset

In [ ]:
report = Report(metrics=[
    RecsysPreset(
        k=5,
        display_features=['action', 'adventure', 'animation'],
        item_features=item_features,
        user_bias_columns=['age', 'gender'],
        item_bias_columns=['moive_age', 'crime'],
    )
])
column_mapping = ColumnMapping(recommendations_type='rank', target='rating', prediction='rank', item_id='title', user_id='user_id')
report.run(
    reference_data=most_popular_df.dropna(subset=['title', 'user_id']).fillna(0),
    current_data=als_df.dropna(subset=['title', 'user_id']).fillna(0),
    column_mapping=column_mapping,
    additional_data={'current_train_data': train.dropna(subset=['title', 'user_id'])}
  )
report

# Tests

In [ ]:
from evidently.test_suite import TestSuite
from evidently.tests import *

In [ ]:
tests = TestSuite(tests=[
    TestPrecisionTopK(k=5),
    TestRecallTopK(k=5),
    TestFBetaTopK(k=5),
    TestHitRateK(k=5),
    TestMAPK(k=5),
    TestMRRK(k=5),
    TestNDCGK(k=5),
    TestNovelty(k=5),
    TestPersonalization(k=5),
    TestSerendipity(k=5, item_features=item_features),
    TestDiversity(k=5, item_features=item_features),
    TestARP(k=5),
    TestGiniIndex(k=5),
    TestCoverage(k=5),
])

column_mapping = ColumnMapping(recommendations_type='rank', target='rating', prediction='rank', item_id='title', user_id='user_id')
tests.run(
    reference_data=most_popular_df.dropna(subset=['title', 'user_id']).fillna(0),
    current_data=als_df.dropna(subset=['title', 'user_id']).fillna(0),
    column_mapping=column_mapping,
    additional_data={'current_train_data': train.dropna(subset=['title', 'user_id'])}
  )
tests

# Test Preset

In [ ]:
from evidently.test_preset import RecsysTestPreset

In [ ]:
tests = TestSuite(tests=[
    RecsysTestPreset(k=5)
])

column_mapping = ColumnMapping(recommendations_type='rank', target='rating', prediction='rank', item_id='title', user_id='user_id')
tests.run(
    reference_data=most_popular_df.dropna(subset=['title', 'user_id']).fillna(0),
    current_data=als_df.dropna(subset=['title', 'user_id']).fillna(0),
    column_mapping=column_mapping,
    additional_data={'current_train_data': train.dropna(subset=['title', 'user_id'])}
  )
tests

# Metric and test defined for recommendation_type = 'scores' only

In [ ]:
# collect data with scores
als_df_scores = pd.DataFrame(
    {
        'user_id': np.repeat(test.user_id.unique(), 30),
        'movie_id': ids.flatten(),
        'scores': scores.flatten()
    }
)
als_df_scores = als_df_scores.merge(test, on=['user_id', 'movie_id'], how='outer')
als_df_scores['scores'] = als_df_scores['scores'].fillna(0)

In [ ]:
from evidently.metrics import ScoreDistribution

In [ ]:
report = Report(metrics=[
  ScoreDistribution(k=5)
])
column_mapping = ColumnMapping(recommendations_type='score', target='rating', prediction='scores', item_id='title', user_id='user_id')
report.run(
    reference_data=None,
    current_data=als_df_scores.dropna(subset=['movie_id', 'user_id']).fillna(0),
    column_mapping=column_mapping,
)
report

In [ ]:
tests = TestSuite(tests=[TestScoreEntropy(k=5, gt=3)])
column_mapping = ColumnMapping(recommendations_type='score', target='rating', prediction='scores', item_id='title', user_id='user_id')
tests.run(
    reference_data=None,
    current_data=als_df_scores.dropna(subset=['movie_id', 'user_id']).fillna(0),
    column_mapping=column_mapping,
)
tests